 # Planetesimal with constant material properties

This Jupyter notebook contains an example case where the mantle has
constant material properties.

First we import the required `pytesimal` modules. To allow this
example to run without the package installed, we import these
modules from the `context.py` script, which adds the package
directory to the python path. This isn't required once the
package is installed; instead modules can be loaded with
`import pytesimal.numerical_methods` etc.

In [ ]:
import matplotlib

# Roundabout import without package installed
from context import setup_functions
from context import load_plot_save
from context import numerical_methods
from context import analysis
from context import core_function
from context import mantle_properties

# View plots within the notebook
%matplotlib inline

We want to use the default parameter set, so we will produce a
default parameter file:

In [ ]:
load_plot_save.make_default_param_file()

If you wanted to change the input parameters, you could now open
and edit this file, replacing the default values with parameters
that suit the planetesimal you wish to model. As we are using
the default parameters, we will just load the parameter file
without editing it:

In [ ]:
(run_ID, folder, timestep, r_planet, core_size_factor,
reg_fraction, max_time, temp_core_melting, olivine_cp,
olivine_density, cmb_conductivity, core_cp, core_density,
temp_init, temp_surface, core_temp_init, core_latent_heat,
kappa_reg, dr, cond_constant, density_constant,
heat_cap_constant) = load_plot_save.load_params_from_file()

For this example, we won't save any outputs. If you're working
on your local machine, you can specify the folder you want to
save your outputs to, and check that this folder exists on your
machine (if it doesn't, a folder will be created):

    folder = 'workflow'
    load_plot_save.check_folder_exists(folder)

This folder can also be specified in the parameters file.
Any variable loaded from the parameters file can be
overwritten before the model runs, which is useful if looping over
a parameter space.

The `setup_functions.set_up()` function creates empty arrays to
be filled with resulting temperatures:

In [23]:
(r_core,
radii,
core_radii,
reg_thickness,
where_regolith,
times,
mantle_temperature_array,
core_temperature_array) = setup_functions.set_up()

# We define an empty list of latent heat that will
# be filled as the core freezes
latent = []

We instantiate the core object. This will keep track of the
temperature of the core as the model runs, cooling as heat
is extracted across the core-mantle boundary. This simple
eutectic core model doesn't track inner core growth, but
this is still a required argument to allow for future
incorporation of more complex core objects:

In [ ]:
core_values = core_function.IsothermalEutecticCore(initial_temperature=core_temp_init, melting_temperature=temp_core_melting,
                                                            outer_r=r_core, inner_r=0, rho=core_density, cp=core_cp,
                                                            core_latent_heat=core_latent_heat)

We define the mantle properties. The default is to have constant
values, so we don't require any arguments for this example:

In [ ]:
(mantle_conductivity, mantle_heatcap, mantle_density) = mantle_properties.set_up_mantle_properties()


We set up the boundary conditions for the mantle. For this example,
we're using fixed temperature boundary conditions at both the
surface and the core-mantle boundary.

In [ ]:
top_mantle_bc = numerical_methods.surface_dirichlet_bc
bottom_mantle_bc = numerical_methods.cmb_dirichlet_bc

Now we let the temperature inside the planestesimal evolve.
This will take a minute or two!

In [ ]:
(mantle_temperature_array,
core_temperature_array,
latent,
)= numerical_methods.discretisation(
    core_values,
    latent,
    temp_init,
    core_temp_init,
    top_mantle_bc,
    bottom_mantle_bc,
    temp_surface,
    mantle_temperature_array,
    dr,
    core_temperature_array,
    timestep,
    r_core,
    radii,
    times,
    where_regolith,
    kappa_reg,
    mantle_conductivity,
    mantle_heatcap,
    mantle_density)


Once that's finished running, we can check that the results
have been cast to the empty array we defined earlier:

In [ ]:
mantle_temperature_array

Now we can use the `pytesimal.analysis` module to find out more
about the model run. We can check when the core was freezing,
so we can compare this to the cooling history of meteorites
and see whether they can be expected to record magnetic remnants
of a core dynamo:

In [ ]:
(core_frozen,
 times_frozen,
 time_core_frozen,
 fully_frozen) = analysis.core_freezing(core_temperature_array,
                                        max_time,
                                        times,
                                        latent,
                                        temp_core_melting,
                                        timestep)


We can calculate arrays of cooling rates from the temperature
arrays:

In [ ]:
mantle_cooling_rates = analysis.cooling_rate(mantle_temperature_array, timestep)
core_cooling_rates = analysis.cooling_rate(core_temperature_array, timestep)

In [31]:
# mantle_cooling_rates = dT_by_dt
# core_cooling_rates = dT_by_dt_core

We can use data from meteorites (cloudy-zone particle diameter)
to estimate their cooling rates in K/s:

In [ ]:
d_im = 147  # cz diameter in nm
d_esq = 158  # cz diameter in nm

imilac_cooling_rate = analysis.cooling_rate_to_seconds(
    analysis.cooling_rate_cloudyzone_diameter(d_im))
esquel_cooling_rate = analysis.cooling_rate_to_seconds(
    analysis.cooling_rate_cloudyzone_diameter(d_esq))

We can use this cooling rate information to find out how
deep these meteorites originally formed, and when they passed
through the temperature of tetrataenite formation (when magnetism
can be recorded):

In [ ]:
im_depth, im_x, im_time_core_frozen, im_Time_of_Crossing, im_Critical_Radius = analysis.meteorite_depth_and_timing(
    imilac_cooling_rate,
    mantle_temperature_array,
    mantle_cooling_rates,
    radii,
    r_planet,
    core_size_factor,
    time_core_frozen,
    fully_frozen,
    dr=1000,
)

esq_depth, esq_x, esq_time_core_frozen, esq_Time_of_Crossing, esq_Critical_Radius = analysis.meteorite_depth_and_timing(
    esquel_cooling_rate,
    mantle_temperature_array,
    mantle_cooling_rates,
    radii,
    r_planet,
    core_size_factor,
    time_core_frozen,
    fully_frozen,
    dr=1000,
)

meteorite_results_dict = { 'Esq results':
                               {'depth': esq_depth,
                                'text result': esq_x},
                           'Im results':
                               {'depth' : im_depth,
                                'text result': im_x,
                                'critical radius': im_Critical_Radius}}

We can print out this information:

In [ ]:
print(f"Imilac depth: {im_depth}; Imilac timing: {im_x}")
print(f"Esquel depth: {esq_depth}; Esquel timing: {esq_x}")

We can plot the cooling history of the planetesimal:

In [ ]:
# define the figure height and width:

fig_w = 6
fig_h = 9

load_plot_save.two_in_one(
    fig_w,
    fig_h,
    mantle_temperature_array,
    core_temperature_array,
    mantle_cooling_rates,
    core_cooling_rates,)

The results can then be saved to the folder specified earlier,
in the form of a `json` file with parameters and meteorite results,
and/or as compressed numpy arrays of temperatures and cooling rates
(which can be loaded and analysed at a later point).

The `json` results file is in the same format as the input
parameters file, and can be loaded to reproduce the same model run.

To save the results, run:

```python
result_filename = 'workflow_results1'

load_plot_save.save_params_and_results(result_filename, run_ID, folder, timestep, r_planet, core_size_factor,
                        reg_fraction, max_time, temp_core_melting, olivine_cp,
                        olivine_density, cmb_conductivity, core_cp, core_density,
                        temp_init, temp_surface, core_temp_init, core_latent_heat,
                        kappa_reg, dr, cond_constant, density_constant,
                        heat_cap_constant, time_core_frozen, fully_frozen,
                        meteorite_results=meteorite_results_dict)


load_plot_save.save_result_arrays(result_filename,
                       folder,
                       mantle_temperature_array,
                       core_temperature_array,
                       mantle_cooling_rates,
                       core_cooling_rates)
```
